# Dynamic
## Dynamic Prompt

In [ ]:
import { SqlDatabase } from "@langchain/classic/sql_db";
import { DataSource } from "typeorm";

const datasource = new DataSource({
    type: "sqlite",
    database: "./Chinook.db", // Replace with the link to your database
});
const db = await SqlDatabase.fromDataSourceParams({
    appDataSource: datasource,
});

In [ ]:
import { tool } from "langchain";
import { z } from "zod";

export const executeSQL = tool(({ query }) => {
    return db.run(query)
}, {
    name: "execute_sql",
    description: "Execute a SQLite command and return results.",
    schema: z.object({ query: z.string() })
})

In [ ]:
import { z } from "zod";

const RuntimeContext = z.object({
    isEmployee: z.boolean()
});
type RuntimeContext = z.infer<typeof RuntimeContext>;

In [ ]:
const SYSTEM = `You are a careful SQLite analyst.

Rules:
- Think step-by-step.
- When you need data, call the tool \`execute_sql\` with ONE SELECT query.
- Read-only only; no INSERT/UPDATE/DELETE/ALTER/DROP/CREATE/REPLACE/TRUNCATE.
- Limit to 5 rows unless the user explicitly asks otherwise.
%s
- If the tool returns 'Error:', revise the SQL and try again.
- Prefer explicit column lists; avoid SELECT *.`

In [ ]:
import { format } from "node:util";
import { dynamicSystemPrompt, type ModelRequest, type AgentState, type Runtime } from "langchain";

const dynamicSystemPromptMiddleware = dynamicSystemPrompt((request: ModelRequest, state: AgentState, runtime: Runtime<RuntimeContext>) => {
    const limits = !runtime.context.isEmployee
        ? "- Limit access to these tables: Album, Artist, Genre, Playlist, PlaylistTrack, Track."
        : ""
    return {
        ...request,
        systemPrompt: format(request.systemPrompt, limits)
    }
})

In [ ]:
import "dotenv/config";
import { createAgent } from "langchain";

const agent = createAgent({
    model: "openai:gpt-5",
    tools: [executeSQL],
    systemPrompt: SYSTEM,
    contextSchema,
    middleware: [dynamicSystemPromptMiddleware]
})

In [ ]:
import { HumanMessage } from "langchain";

const question = new HumanMessage("What is the most costly purchase by Frank Harris?");

const result = await agent.invoke({
    messages: [question]
}, {
    context: {
        first: "Frank",
        last: "Harris"
    }
});

result.messages.at(-1).content;